In [ ]:
import pandas as pd
import numpy as np
from keras.models import load_model

import re
from time import time, sleep
from os.path import dirname, join

from bokeh.io import curdoc
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import PreText, Select

from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
from bokeh.palettes import Viridis256, Inferno256, Plasma256
from bokeh.models import LogColorMapper, LogTicker, ColorBar
from bokeh.models import ColorMapper
from bokeh.layouts import row, column, gridplot
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, RangeSlider

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()

In [ ]:
fname = '../storage/latent/latent_space_LHC18fop_100-100-64--10_train-on-all_v0.csv'
df_latent = pd.read_csv(fname, index_col=0)
decoder = load_model(fname.replace('latent_space', 'decoder').replace('.csv', '.h5'))
# df_data = pd.read_csv('../data/trending_merged_LHC18fop_withGraphs.csv')
df = df_latent
df.head()

# Stable

In [ ]:
exec_times = []
def modify_doc(doc, exec_times=exec_times):
    
    histos = []
    sliders = []

    for col in df.columns:
        if not col.startswith('lat'): 
            continue

        counts, edges = np.histogram(df[col], density=False, bins=50)

        f = figure(plot_width=300, plot_height=60, tools='', 
#                    active_drag="xbox_select", 
                   y_axis_type="log", y_range=(3e-1, 1e4)
                  )
        h = f.quad(top=counts, bottom=1e-6, left=edges[:-1], right=edges[1:],
                   fill_color="navy", line_color="white", alpha=0.5)

        vmin, vmax = df[col].min(), df[col].max()
        vrange = vmax - vmin
        vmin, vmax = np.floor(vmin)-0.1*vrange, np.ceil(vmax)+0.1*vrange
        s = Slider(start=vmin, end=vmax, value=0.5*vmin+0.5*vmax, step=min(1, (vmax-vmin)/100), 
#                         default_size=600
                        callback_policy='mouseup',
#                         callback_throttle=500,
                       )
        
        s.name = col
        histos.append(f)
        sliders.append(s)
   

    reco_fig = figure(plot_width=1000, plot_height=200, tools='', 
#                    active_drag="xbox_select", 
#                    y_axis_type="log", 
               y_range=(-10, 10)
               )
    reco_bars = reco_fig.quad(top= np.random.randn(88), bottom=0, left=range(88), right=range(1,89),
               fill_color="blue", line_color="red", alpha=0.3)
    select_bars = reco_fig.quad(top= np.random.randn(88), bottom=0, left=range(88), right=range(1,89),
               fill_color="red", line_color="red", alpha=0.9)
    
    
    def update(attr, old, new):
        tic = time()
        latent_vec = np.array([s.value for s in sliders])
        latent_vec = latent_vec.reshape(1, latent_vec.shape[0])
        preds = decoder.predict(latent_vec)[0]
#         print(preds)
        reco_bars.data_source.data['top'] = preds
        preds_select = np.zeros_like(preds)
        preds_select[81] = preds[81]
        select_bars.data_source.data['top'] = preds_select
#         print(reco_bars.data_source.data['top'])
        exec_times.append(time() - tic)


    for s in sliders:
        s.on_change('value', update)

    update(0,0,0)
        
    # # # layout    
    n_cols = 3
    rows = []
    for i in range(int(np.ceil(len(histos)/3))):
        idx_low = i*n_cols
        idx_high = min((i+1)*n_cols, len(histos))
        r = row([column(f,s) for f,s in zip(histos[idx_low:idx_high], sliders[idx_low:idx_high])])
        rows.append(r)
        
    rows.append(reco_fig)
    layout = column(rows)

    doc.add_root(layout)
    
handler = FunctionHandler(modify_doc)
app = Application(handler)

show(app)

In [ ]:
exec_times_arr = np.array(exec_times)
print(f'exec. time of `upgrade` = {np.mean(exec_times_arr*1000):.1f} +/- {np.std(exec_times_arr*1000):.1f} ms \t({len(exec_times_arr)} runs) ')

# Dev

In [ ]:
exec_times = []
def modify_doc(doc, exec_times = exec_times):
    
    SELECTED_BAR_IDX = 81
    
    histos = []
    sliders = []
    markers = []
    
    for col in df.columns:
        if not col.startswith('lat'): 
            continue

        counts, edges = np.histogram(df[col], density=False, bins=50)

        f = figure(plot_width=300, plot_height=60, tools='', 
#                    active_drag="xbox_select", 
                   y_axis_type="log", y_range=(3e-1, 1e4)
                  )
        h = f.quad(top=counts, bottom=1e-6, left=edges[:-1], right=edges[1:],
                   fill_color="navy", line_color="white", alpha=0.5)

        vmin, vmax = df[col].min(), df[col].max()
        vrange = vmax - vmin
        vmin, vmax = np.floor(vmin)-0.1*vrange, np.ceil(vmax)+0.1*vrange
        s = Slider(start=vmin, end=vmax, value=0.5*vmin+0.5*vmax, step=min(1, (vmax-vmin)/100), 
#                         default_size=600
                        callback_policy='mouseup',
#                         callback_throttle=500,
                       )
        m = f.triangle([0,], [5e3,], fill_color='orange', line_color='red', size=8, angle=3.14)
        
        s.name = col
        histos.append(f)
        sliders.append(s)
        markers.append(m)
   

    reco_fig = figure(plot_width=950, plot_height=200, tools='', 
               y_range=(-10, 10)
               )
    reco_bars = reco_fig.quad(top= np.random.randn(88), bottom=0, left=range(88), right=range(1,89),
               fill_color=["blue" for _ in range(88)], line_color="white", alpha=[0.3+i/100 for i in range(88)])
#     select_bars = reco_fig.quad(top= np.random.randn(88), bottom=0, left=range(88), right=range(1,89),
#                fill_color="red", line_color="red", alpha=0.9)
    
    
    def update(attr, old, new):
        tic = time()
        latent_vec = np.array([s.value for s in sliders])
        latent_vec = latent_vec.reshape(1, latent_vec.shape[0])
        preds = decoder.predict(latent_vec)[0]
#         print(preds)
        reco_bars.data_source.data['top'] = preds
#         preds_select = np.zeros_like(preds)
#         preds_select[SELECTED_BAR_IDX] = preds[SELECTED_BAR_IDX]
#         select_bars.data_source.data['top'] = preds_select
        reco_bars.data_source.data['fill_color'] = ['blue' if idx != SELECTED_BAR_IDX else 'red' for idx in range(88)]
        reco_bars.data_source.data['fill_alpha'] = [0.2 if idx != SELECTED_BAR_IDX else 0.9 for idx in range(88)]
#         print(reco_bars.data_source.data['top'])

        for s,m in zip(sliders, markers):
            val = s.value
            if val != new: 
                # skip markers which sliders hasn't changed
                continue 
            m.data_source.data['x'] = [val,]
#             print(f'value of {m} set to {val}')
        exec_times.append(time()-tic)
#         print(reco_bars.data_source.data['fill_alpha'])
#         print('up')
#         sleep(0.05)  # upper limit for exec time is ~50 ms



    for s in sliders:
        s.on_change('value', update)

    update(0,0,0)
        
    # # # layout    
    n_cols = 3
    rows = []
    for i in range(int(np.ceil(len(histos)/3))):
        idx_low = i*n_cols
        idx_high = min((i+1)*n_cols, len(histos))
        r = row([column(f,s) for f,s in zip(histos[idx_low:idx_high], sliders[idx_low:idx_high])])
        rows.append(r)
        
    rows.append(reco_fig)
    layout = column(rows)

    doc.add_root(layout)
    
handler = FunctionHandler(modify_doc)
app = Application(handler)

show(app)

In [ ]:
exec_times_arr = np.array(exec_times)
print(f'exec. time of `upgrade()` = {np.mean(exec_times_arr*1000):.1f} +/- {np.std(exec_times_arr*1000):.1f} ms \t({len(exec_times_arr)} runs) ')